In [67]:
import xml.etree.ElementTree
import math
import numpy as np
from scipy.spatial.transform import Rotation as R

In [68]:
def rotation_matrix(axis, theta):
    """
    Return the rotation matrix associated with counterclockwise rotation about
    the given axis by theta radians.
    """
    axis = np.asarray(axis)
    axis = axis / math.sqrt(np.dot(axis, axis))
    a = math.cos(theta / 2.0)
    b, c, d = -axis * math.sin(theta / 2.0)
    aa, bb, cc, dd = a * a, b * b, c * c, d * d
    bc, ad, ac, ab, bd, cd = b * c, a * d, a * c, a * b, b * d, c * d
    return np.array([[aa + bb - cc - dd, 2 * (bc + ad), 2 * (bd - ac)],
                     [2 * (bc - ad), aa + cc - bb - dd, 2 * (cd + ab)],
                     [2 * (bd + ac), 2 * (cd - ab), aa + dd - bb - cc]])

In [73]:

tree = xml.etree.ElementTree.parse("generated/generatedURDF.urdf")

#Normalize all joint positions
for joint in tree.getroot().findall("./joint"):
    origin = joint.find('./origin')
    old_xyz = [float(i) for i in origin.attrib['xyz'].split(' ')]
    new_xyz = [0.75*i for i in old_xyz]
    origin.attrib['xyz'] = ' '.join([str(i) for i in new_xyz])
#print(xml.etree.ElementTree.tostring(tree.getroot()).decode())

#Remove all visual geometries
for link in tree.getroot().findall("./link"):
    if link.find('./visual') is not None:
        for visualElement in link.findall('./visual'):
            link.remove(visualElement)

sphere_string = lambda radius: f'<sphere radius="{radius}"></sphere>'
def add_shape(link_element, shape_string): 
    link_element.append(xml.etree.ElementTree.fromstring(f"<visual><geometry>{shape_string}</geometry></visual>"))
#Add cylinders that go from the parent to the child
for link in tree.getroot().findall("./link"):
    
    #Find joint for which the link is the child, and the link for which the link is the parent
    link_name = link.get('name')
    
    parentOfJoints = tree.getroot().findall(f"./joint/parent[@link='{link_name}']/..")
    if len(parentOfJoints) == 0:
        add_shape(link,sphere_string(0.025))
        print(f"Link: {link_name} skipped since its not a parent of any joint")
        continue
    for parentOfJoint in parentOfJoints:
        print(f"""
    Link: {link_name}
        Parent Link in Joint {parentOfJoint.get('name') if parentOfJoint is not None else "None"}
    """.strip())
        
        get_position_from_joint = lambda joint: np.array([float(e) for e in joint.find('./origin').get('xyz').split()])
        
        length = sum([(x)**2 for x in get_position_from_joint(parentOfJoint)])**0.5
        

        parent_of_joint_position = get_position_from_joint(parentOfJoint)
        if np.linalg.norm(parent_of_joint_position) == 0:
            print(f"Link {link_name} same position as its parent")
            add_shape(link,sphere_string(0.025))
            continue

        plane_normal = np.cross(np.array([0,0,1]),parent_of_joint_position)
        dot = np.dot(parent_of_joint_position, np.array([0,0,1]))
        phi = np.arccos(dot/np.linalg.norm(parent_of_joint_position))

        rotation_vector = phi*(plane_normal/np.linalg.norm(plane_normal)) if np.linalg.norm(plane_normal) != 0 else np.array([0,0,0])
        rotation_eulers = R.from_rotvec(rotation_vector).as_euler("xyz")

        roll = rotation_eulers[0]
        pitch = rotation_eulers[1]
        yaw = rotation_eulers[2]
        rpy = f"{roll} {pitch} {yaw}"
        
        

        new_xyz = [(i/2) for i in parent_of_joint_position]
        link.append(xml.etree.ElementTree.fromstring(f"""<visual>
                                                        <geometry>
                                                            <cylinder length="{length}" radius="0.01"></cylinder>
                                                        </geometry>
                                                        <origin xyz = '{' '.join([str(i) for i in new_xyz])}' rpy = '{rpy}'>
                                                        </origin>
                                                    </visual>"""))


Link: base
        Parent Link in Joint floating_base
Link base same position as its parent
Link: trunk
        Parent Link in Joint FR_hip_joint
Link: trunk
        Parent Link in Joint FL_hip_joint
Link: trunk
        Parent Link in Joint RR_hip_joint
Link: trunk
        Parent Link in Joint Spinal Midpoint Lower
Link: trunk
        Parent Link in Joint Spinal Midpoint Upper
Link: trunk
        Parent Link in Joint RL_hip_joint
Link: FR_hip
        Parent Link in Joint FR_thigh_joint
Link: FR_thigh
        Parent Link in Joint FR_calf_joint
Link: FR_calf
        Parent Link in Joint FR_foot_fixed
Link: FR_foot skipped since its not a parent of any joint
Link: FL_hip
        Parent Link in Joint FL_thigh_joint
Link: FL_thigh
        Parent Link in Joint FL_calf_joint
Link: FL_calf
        Parent Link in Joint FL_foot_fixed
Link: FL_foot skipped since its not a parent of any joint
Link: RR_hip
        Parent Link in Joint RR_thigh_joint
Link: RR_thigh
        Parent Link in Joint RR_ca

In [74]:
tree.write("generated/sim_ready_robot.urdf", encoding="utf-8", xml_declaration=True)